# Reprotrack - Interactive session
For this session, we have collected five different reproducibility/replication challenges that you can solve in any order you want. For the challenges we have provided you with a subsample of the Geolife dataset but you can also use your own data. 

1. Tracking quality assessment
2. Travel mode split
3. Replicate mobility scaling laws - Jump length
4. Replicate mobility scaling laws - Radius of gyration
5. Context integration with OSM data



# Import dependencies and libraries

In [1]:
import os
import json
import pickle as pickle

import geopandas as gpd
import pandas as pd
import numpy as np

# for fitting empirical distributions for radius of gyration and jump length
import powerlaw
# for fetching osm data
import osmnx as ox

# for visualization
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# trackintel -> the functions will be imported with full names
import trackintel as ti


In [2]:
# options for better printing and visualizing

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

matplotlib.rcParams["figure.dpi"] = 300
matplotlib.rcParams["xtick.labelsize"] = 13
matplotlib.rcParams["ytick.labelsize"] = 13
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

%matplotlib inline

In [3]:
# get the data dir from the config file -> default stored in Data/Geolife/Data folder

DATA_DIR = os.path.join("..", "paths.json")
with open(DATA_DIR) as json_file:
    CONFIG = json.load(json_file)

save_dir = os.path.join("..", CONFIG["data_dir"])

# Load Geolife data
Trackintel provides a function [read_geolife](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_geolife) to directly load [Geolife](https://www.microsoft.com/en-us/research/publication/geolife-gps-trajectory-dataset-user-guide/) dataset into the trackintel positionfixes. For the tutorial and interactive section, we provided you a sample dataset with 20 selected users from the geolife dataset. This can be directly loaded from the `save_dir`. 

In [4]:

pfs, mode_labels = ti.io.read_geolife(save_dir, print_progress=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:28<00:00,  1.42s/it]


## Reading your own data
Trackintel provides an I/O module for accessing movement data and storing intermediate or final results in a file or database. Three methods for converting movement data with attached attribute information to Trackintel-compatible formats are provided:
1) reading from Pandas Dataframes and Geopandas Geodataframes (**recommended**)
2) reading and writing from CSV file formats
3) reading and storing from PostgreSQL databases with PostGIS extension. Check the [input/output](https://trackintel.readthedocs.io/en/latest/modules/io.html) module for more information. 

An important consideration before loading data is to find the appropriate movement level for your data. Depending on the semantics of the data, you should call different reading functions, e.g., [read_positionfixes_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_positionfixes_csv) or [read_staypoints_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_staypoints_csv). We provide a reference of the different data types in the figure below:

![Trackintel data support](prepare/trackintel_structure.png)

In [5]:
# TODO: use ti.read_positionfixes_csv() or ti.read_staypoints_csv() for loading your data. Check the documentation for the required input parameters
# selected_pfs = ti.read_positionfixes_csv()

# 1. Calculate tracking quality
Trackintel provides functions to assist in examining the tracking quality of users from the temporal perspective. We use temporal tracking coverage, defined as the proportion of time the user's whereabouts are recorded, to measure tracking quality. As positionfixes contain only absolute timestamps for each record, one needs to define how long a time gap shall be to regard a user's whereabouts are missing. This design choice can be specified when aggregating positonfixes into staypoints and trips, using the respective temporal parameters. 

## Generate staypoints and trips 
Select the temporal parameters to reflect your design choice of a `time gap`.

Tasks: 
- create staypoints
- add activities to staypoints
- create triplegs
- create trips

In [6]:
# generate staypoints, triplegs and trips
pfs, sp = pfs.generate_staypoints(gap_threshold=24 * 60, include_last=True, print_progress=True, dist_threshold=200, time_threshold=30, n_jobs=-1)

# create activity flag
sp = sp.create_activity_flag(method="time_threshold", time_threshold=25)

# generate triplegs
pfs, tpls = pfs.generate_triplegs(sp, gap_threshold=15, print_progress=True)

# generate trips
sp, tpls, trips = sp.generate_trips(tpls, add_geometry=False)

C:\ProgramData\Anaconda3\envs\reprotrack\lib\site-packages\trackintel\preprocessing\positionfixes.py:113: UserWarning: 1690 duplicates were dropped from your positionfixes. Dropping duplicates is recommended but can be prevented using the 'exclude_duplicate_pfs' flag.
  warnings.warn(warn_str)
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.50it/s]

KeyboardInterrupt



## Calculate tracking quality
We then concatenate staypoints and triplegs to obtain a continues timeline for measuring the tracking coverage.  

In [ ]:
# merge trips and staypoints
df_all = pd.concat([sp, trips])


Trackintel provides the [trackintel.analysis.temporal_tracking_quality](https://trackintel.readthedocs.io/en/latest/modules/analysis.html#trackintel.analysis.temporal_tracking_quality) function to measure the temporal tracking coverage. For the function to work, users need to specify a `granularity` (i.e., to what time extent is the tracking data compared to?). Check the documentations and decide on a desired granularity.

### Overall quality

In [ ]:
# get quality
total_quality = ti.analysis.temporal_tracking_quality(df_all, granularity="all")

boxplot = total_quality.boxplot(column="quality", figsize=(5, 5))

boxplot.set_xticklabels([])

plt.xlabel("Tracking quality", fontsize=16)
plt.ylabel("Proportion", fontsize=16)
plt.show()

### Quality by week

In [ ]:
# get quality
week_quality = ti.analysis.temporal_tracking_quality(df_all, granularity="week")

week_quality_user = week_quality.loc[week_quality["quality"] != 0].groupby("week_monday", as_index=False)["quality"].median()

# construct the index
week_quality_user.set_index("week_monday",inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(week_quality_user.index, week_quality_user['quality'], label="Geolife subset", color=colors[0])
ax.set_ylabel("Tracking Quality by Month (Median)", fontsize=16)
ax.set_xlabel("", fontsize=16)

ax.xaxis.set_minor_locator(mdates.DayLocator(interval=14))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=9))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))

ax.tick_params(axis='x', which='major', labelsize=12)

ax.set_ylim([0, 1.09])
ax.legend(prop={'size': 12}, loc = 'upper right')

plt.show()

### Quality by day

In [ ]:
# get quality
day_quality = ti.analysis.temporal_tracking_quality(df_all, granularity="day")

day_quality_user = day_quality.loc[day_quality["quality"] != 0].groupby("day", as_index=False)["quality"].median()
# construct the index
day_quality_user.set_index("day",inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(day_quality_user.index, day_quality_user['quality'], label="Geolife subset", color=colors[0])
ax.set_ylabel("Tracking Quality by Day (Median)", fontsize=16)
ax.set_xlabel("", fontsize=16)

ax.xaxis.set_minor_locator(mdates.DayLocator(interval=14))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=9))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

ax.tick_params(axis='x', which='major', labelsize=12)

ax.set_ylim([0, 1.09])
ax.legend(prop={'size': 12}, loc = 'upper right')

plt.show()

### Quality by weekday

In [ ]:
# get quality
weekday_quality = ti.analysis.temporal_tracking_quality(df_all, granularity="weekday")

weekday_quality_user = weekday_quality.loc[weekday_quality["quality"] != 0].groupby("weekday", as_index=False)["quality"].median()

weekday_quality_user.set_index("weekday",inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(weekday_quality_user.index, weekday_quality_user['quality'], label="Geolife subset", color=colors[0])
ax.set_ylabel("Tracking Quality by Weekday (Median)", fontsize=15)
ax.set_xlabel("", fontsize=15)

ax.tick_params(axis='x', which='major', labelsize=12)

ax.set_ylim([0, 1.09])
ax.set_xticks(np.arange(7), labels= ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

ax.legend(prop={'size': 12}, loc = 'upper right')

plt.show()

### Quality by hour

In [ ]:
# get quality
hour_quality = ti.analysis.temporal_tracking_quality(df_all, granularity="hour")

hour_quality_user = hour_quality.loc[hour_quality["quality"] != 0].groupby("hour", as_index=False)["quality"].median()

hour_quality_user.set_index("hour",inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(hour_quality_user.index, hour_quality_user['quality'], label="Geolife subset", color=colors[0])
ax.set_ylabel("Tracking Quality by Hour (Median)", fontsize=15)
ax.set_xlabel("", fontsize=15)

ax.tick_params(axis='x', which='major', labelsize=12)

ax.set_ylim([0, 1.09])
ax.set_xticks(np.arange(24))
ax.set_xlabel("Hour of the day", fontsize=15)

ax.legend(prop={'size': 12}, loc = 'upper right')

plt.show()

# 2. Calculate the travel mode split
Travel mode choices are formed due to the everyday needs and constraints of individuals and are influenced by travel-related factors such as cost, time, accessibility, and comfort. It is, therefore, not uncommon for mobility systems to be evaluated and compared based on the current modal splits of the overall population. Trackintel provides a series of travel mode-related functions to assist in mode analysis of individual movement. 

## Attach mode labels to triplegs - true labels and mode detection based on speed
- Attach labels provided by Geolife using the Trackintel function [geolife_add_modes_to_triplegs](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.geolife_add_modes_to_triplegs)
- Impute mode labels based on speed using the Trackintel function [predict_transport_mode](geolife_add_modes_to_triplegs)

In [ ]:
# assign the true mode labels to triplegs, with the help of trackintel functions 
tpls = ti.io.geolife_add_modes_to_triplegs(tpls, selected_mode_labels)



In [ ]:
# impute the mode of the other triplegs based on speed 
tpls["pred_mode"] = ti.analysis.predict_transport_mode(tpls)["mode"]
tpls.loc[tpls["mode"].isna(), "mode"] = tpls.loc[tpls["mode"].isna(), "pred_mode"]
tpls.drop(columns={"pred_mode"}, inplace=True)

In [ ]:
mode_to_category = {
    "fast_mobility": "fast",
    "car": "motorized",
    "bus": "motorized",
    "taxi": "motorized",
    "ecar": "motorized",
    "motorized_mobility": "motorized",
    "bike": "slow",
    "walk": "slow",
    "slow_mobility": "slow",
}

# change mode labels with the above mapping
tpls["mode"] = tpls["mode"].map(mode_to_category)
tpls.dropna(subset=["mode"], inplace=True)
tpls["mode"].value_counts()

## Visualize modal split

In [ ]:
# count
modal_split_count = ti.analysis.calculate_modal_split(tpls, freq="M", metric="count", per_user=False, norm=True)

fig, ax = plt.subplots(figsize=(10, 6))
ax = ti.plot_modal_split(
    modal_split_count,
    date_fmt_x_axis="%Y-%m",
    y_label="Percentage of count",
    skip_xticks=2,
    n_col_legend=4,
    fig=fig,
    axis=ax,
    borderaxespad=2,
)
fig.autofmt_xdate()

In [ ]:
# distance
modal_split_distance = ti.analysis.calculate_modal_split(tpls, freq="M", metric="distance", per_user=False, norm=True)

fig, ax = plt.subplots(figsize=(10, 6))
ax = ti.plot_modal_split(
    modal_split_distance,
    date_fmt_x_axis="%Y-%m",
    y_label="Percentage of distance",
    skip_xticks=2,
    n_col_legend=4,
    fig=fig,
    axis=ax,
    borderaxespad=2,
)
fig.autofmt_xdate()

In [ ]:
# duration
modal_split_duration = ti.analysis.calculate_modal_split(tpls, freq="M", metric="duration", per_user=False, norm=True)

fig, ax = plt.subplots(figsize=(10, 6))
ax = ti.plot_modal_split(
    modal_split_duration,
    date_fmt_x_axis="%Y-%m",
    y_label="Percentage of duration",
    skip_xticks=2,
    n_col_legend=4,
    fig=fig,
    axis=ax,
    borderaxespad=2,
)
fig.autofmt_xdate()

# 3. Calculate Jump length

## Generate staypoints
Jump length measures the distance between consecutive stays. This definition corresponds to trackintel's **staypoint** data model. The following code is to generate staypoints from the raw positionfixes with the trackintel function [generate_staypoints](https://trackintel.readthedocs.io/en/latest/modules/preprocessing.html#trackintel.preprocessing.generate_staypoints). Click on the link to check the documentation on the meaning of the input parameters.

In [ ]:

pfs, sp = selected_pfs.generate_staypoints(gap_threshold=24 * 60, include_last=True, print_progress=True, dist_threshold=200, time_threshold=30, n_jobs=-1)

## Calculate jump length
Trackintel provide implementations for calculating the jumplength with staypoints that have geometry column. Check out the function [trackintel.analysis.metrics.jump_length()](https://trackintel.readthedocs.io/en/latest/modules/analysis.html#trackintel.analysis.jump_length). The output of the function is available as a series that shares the same index as the input dataframe. This ensures that we can use a column to hold the result (e.g., using `sp["jumplength"]`).

In [ ]:
sp["jumplength"] = ti.analysis.jump_length(sp)
jumplength_arr = sp.dropna(subset=["jumplength"])["jumplength"].values

## Visualize with powerlaw library 

In [ ]:
plt.figure(figsize=(8, 5))


# fit power law
xmin = 100
fit = powerlaw.Fit(jumplength_arr[jumplength_arr>xmin], xmin=xmin)

# plotting
powerlaw.plot_pdf(jumplength_arr[jumplength_arr>xmin], label="count")
fit.power_law.plot_pdf(linestyle="--", label="powerlaw fit")
fit.truncated_power_law.plot_pdf(linestyle="--", label="truncated power law")
fit.lognormal.plot_pdf(linestyle="--", label="lognormal fit")

xlabel = "$\Delta r\,(m)$"
ylabel = "$P(\Delta r)$"

plt.legend(prop={"size": 13})
plt.xlabel(xlabel, fontsize=16)
plt.ylabel(ylabel, fontsize=16)

plt.show()

# 4. Calculate Radius of Gyration

- You need staypoints

## Generate staypoints

In [ ]:
pfs, sp = selected_pfs.generate_staypoints(gap_threshold=24 * 60, include_last=True, print_progress=True, dist_threshold=200, time_threshold=30, n_jobs=-1)

## Calculate radius of gyration with trackintel functions

In [ ]:
rg_count = ti.analysis.radius_gyration(sp, method="count")

In [ ]:
sp["duration"] = (sp["finished_at"] - sp["started_at"])
rg_duration = ti.analysis.radius_gyration(sp, method="duration")

In [ ]:
rg_duration

## Visualize with powerlaw package

In [ ]:
plt.figure(figsize=(8, 5))


# fit power law
xmin = 100
fit = powerlaw.Fit(rg_duration, xmin=xmin)

# plotting
powerlaw.plot_pdf(rg_count, label="count")
powerlaw.plot_pdf(rg_duration, label="duration")
fit.power_law.plot_pdf(linestyle="--", label="powerlaw fit of duration")
fit.truncated_power_law.plot_pdf(linestyle="--", label="truncated power law of duration")
fit.lognormal.plot_pdf(linestyle="--", label="lognormal fit of duration")

xlabel = "$Rg$ (m)"
ylabel = "$P(Rg)$"

plt.legend(prop={"size": 13})
plt.xlabel(xlabel, fontsize=16)
plt.ylabel(ylabel, fontsize=16)

plt.show()


# 5. Activity prediction - trajectory context integration with OSM data

## Generate staypoints and locations
In the trackintel definition, staypoints 

In [ ]:
# generate staypoints, triplegs and trips
pfs, sp = selected_pfs.generate_staypoints(gap_threshold=24 * 60, include_last=True, print_progress=True, dist_threshold=200, time_threshold=30, n_jobs=-1)

# create activity flag
sp = sp.create_activity_flag(method="time_threshold", time_threshold=25)

# generate locations
## filter activity staypoints
sp = sp.loc[sp["is_activity"] == True]

# generate locations
sp, locs = sp.generate_locations(
    epsilon=20, num_samples=1, distance_metric="haversine", agg_level="dataset", n_jobs=-1, print_progress=True
)
# filter noise staypoints
valid_sp = sp.loc[~sp["location_id"].isna()]

In [ ]:
# inspect locations
locs.head(5)

## Extract POI from OSM and link with locations

You can imagine that we can make certain statements about the reason for a trip if we know what is in the area at the destination. For example, it makes sense that we go to a university district to study, to a residential area when we go home, or to an area with many restaurants and bars to meet friends.

### POI from OSM

Following this intuition, we extract POIs from OpenStreetMap and link them spatially with the locations. First we define a filter, defining that OSM objects with the attribute `amenity` or `shop` are extracted from the OSM data set. These tags are very widespread in the OSM data because they describe various objects in a relatively general way. You can find out more about the OSM tags at https://wiki.openstreetmap.org/wiki/DE:Key:amenity and https://taginfo.openstreetmap.org.

We use the library `osmnx` to download the newest osm data from the internet and extract certain subsets of the data as GeoDataFrame. `osmnx` is an open-source library that provides support for working with OpenStreetMap data. It enables straightforward fetching of OSM data and is still actively maintained by a group of GIS enthusiasts. Checkout the library [homepage](https://osmnx.readthedocs.io/en/stable/getting-started.html) and its [examples](https://github.com/gboeing/osmnx-examples/tree/main/notebooks).

Here we define the filter below for the POIs for you.

In [ ]:
custom_filter = {'amenity': True, 'shop': True}

# apply the filter
pois = ox.features_from_bbox(40.0920, 39.8594, 116.4410, 116.1152, custom_filter)

# we now want the 'node' types
pois = pois.iloc[pois.index.get_level_values('element_type') == "node"]

pois.shape

Use the function `plot`, which is provided by the `geopandas` library for `GeoDataFrames`.

In [ ]:
pois.plot(figsize=(12, 8))
plt.show()

We can now display this data on a map and inspect it again with `.head (...)`.

In [ ]:
# Gather info about POI type (combines the tag info from "amenity" and "shop").
pois["poi_type"] = pois["amenity"]
pois["poi_type"] = pois["poi_type"].fillna(pois["shop"])

# plot
ax = pois.plot(column='poi_type', markersize=3, figsize=(12,12), legend=True, legend_kwds=dict(loc='upper left', ncol=5, bbox_to_anchor=(1, 1)))
plt.show()

In [ ]:
# Inspect the DataFrame using .head(...), .columns, etc.
pois.columns

### Conversion to projected coordinate system, buffering and spatial join

In the next step, we want to change the coordinate system from WGS 1984 to EPSG:4479, as the latter allows us to quantify distances in meters. `geopandas` offers the function `.to_crs(...)` for this operation. Then create a buffer around the sample points with the function `.buffer(dist)` and use the `gpd.sjoin(...)` function to perform a spatial join between the buffered sample points and the POI. Look at the data with `.head(...)` and check if the results make sense.

In [ ]:
# Create buffer around locations. assume distance of 100
epsilon = 100


# project the data into the coordinate system EPSG:4479.
locs_4479 = gpd.GeoDataFrame(locs.drop(columns="center"), crs="EPSG:4326", geometry="extent")
locs_4479 = locs_4479.to_crs('EPSG:4479')
pois_4479 = pois.to_crs('EPSG:4479')

locs_buffer = locs_4479.copy()

# Create a buffer of reasonable size around the locations.
# Remember to set a distance threshold in .buffer() (now the unit is in meters)
locs_buffer['extent'] = locs_4479.buffer(epsilon)


# Join the two GeoDataFrames using geopanda's sjoin function (use how='left').
locs_pois = gpd.sjoin(locs_buffer, pois_4479, how='left')
locs_pois = locs_pois.reset_index()


In [ ]:
# examine locations
locs_pois.head(3)

We can now count how often a certain type of POI has occurred in the vicinity of a location and draw this in a figure. Attention: We only plot the first 20 locations for simplicity.

In [ ]:
poi_types_per_loc = locs_pois.groupby(['id', 'poi_type'])['id'].count().unstack('poi_type').head(20)
poi_types_per_loc.plot(kind='bar', stacked=True, figsize=(12, 8))
plt.legend(ncol=3, loc='center left', bbox_to_anchor=(1.04, 0.5))
plt.show()